In [10]:
from http.client import ResponseNotReady
from google.cloud import speech_v1 as speech
import os
import json
from google.protobuf.json_format import MessageToJson

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"D:\UPES\Project\Project Auto_Dub\Local_to_GCP\upheld-beach-360516-95be0c166639.json"


In [11]:
def speech_to_text(config, audio):
    client = speech.SpeechClient()
    response = client.recognize(config=config, audio=audio)
    # print(response)
    return response

In [12]:
def print_sentences(response):
    for result in response.results:
        best_alternative = result.alternatives[0]
        transcript = best_alternative.transcript
        confidence = best_alternative.confidence
        print("-" * 80)
        print(f"Transcript: {transcript}")
        print(f"Confidence: {confidence:.0%}")
        print_word_offsets(best_alternative)

In [13]:
def print_word_offsets(alternative):
    for word in alternative.words:
        confidence = word.confidence
        start_s = word.start_time.total_seconds()
        end_s = word.end_time.total_seconds()
        word = word.word
        print(f"{start_s:>7.3f} | {end_s:>7.3f} | {word} |{confidence:.0%}")
        with open('transcribe.txt', 'a') as the_file:
            the_file.write(f"{start_s:>7.3f} | {end_s:>7.3f} | {word} |{confidence:.0%}\n")

In [14]:
config = dict(
    language_code="en-US",
    enable_automatic_punctuation=True,
    enable_word_time_offsets=True,
    audio_channel_count = 2,
    enable_word_confidence=True)

audio = dict(uri="gs://flac_file111/how-to-speak-so-that-people-want-to-listen-julian-treasure_RRYwKciD.flac")

In [15]:
response = speech_to_text(config, audio)
#print(response)


In [16]:
print(type(response))

<class 'google.cloud.speech_v1.types.cloud_speech.RecognizeResponse'>


In [17]:
for i in response.results:
    for word in i.alternatives:
        all_words = []
        for j in word.words:            
            start_time = j.start_time
            end_time = j.end_time
            word = j.word
            confidence = j.confidence
            row_to_add = [start_time, end_time, word, confidence]
            all_words.append(row_to_add)
            
            
print(all_words)
            

[[datetime.timedelta(seconds=18, microseconds=200000), datetime.timedelta(seconds=18, microseconds=700000), "It's", 0.9876290559768677], [datetime.timedelta(seconds=18, microseconds=700000), datetime.timedelta(seconds=18, microseconds=700000), 'the', 0.9876290559768677], [datetime.timedelta(seconds=18, microseconds=700000), datetime.timedelta(seconds=18, microseconds=900000), 'most', 0.9876290559768677], [datetime.timedelta(seconds=18, microseconds=900000), datetime.timedelta(seconds=19, microseconds=300000), 'powerful', 0.9876290559768677], [datetime.timedelta(seconds=19, microseconds=300000), datetime.timedelta(seconds=19, microseconds=400000), 'sound', 0.9379932880401611], [datetime.timedelta(seconds=19, microseconds=400000), datetime.timedelta(seconds=19, microseconds=600000), 'in', 0.9354185461997986], [datetime.timedelta(seconds=19, microseconds=600000), datetime.timedelta(seconds=19, microseconds=700000), 'the', 0.955124020576477], [datetime.timedelta(seconds=19, microseconds=70

In [18]:
import pandas as pd                                          #csv
headers = ['Start Time','End Time', "Word", 'Confidence']
words_df = pd.DataFrame(data = all_words,header = headers)

TypeError: DataFrame.__init__() got an unexpected keyword argument 'header'

In [20]:
with open(r'all_words.csv', 'w') as fp:
    
    str_to_write='start_time, end_time, word, confidence'
    fp.write(str_to_write + '\n')
    
    for row in all_words:
        row_str = []
        for item in row:
            str_item = str(item)
            row_str.append(str_item)
            
        str_to_write = ','.join(row_str)
        # write each item on a new line
        fp.write(str_to_write + '\n')
    print('Done')

Done
